In [1]:
from qiskit_machine_learning.datasets import ad_hoc_data

adhoc_dimension = 2
train_features, train_labels, test_features, test_labels, adhoc_total = ad_hoc_data(
    training_size=60,
    test_size=15,
    n=adhoc_dimension,
    gap=0.3,
    plot_data=False,
    one_hot=False,
    include_sample_total=True,
)

In [2]:
from qiskit_ibm_runtime import Session, SamplerV2, EstimatorV2
from qiskit.providers.fake_provider import Fake127QPulseV1
from qiskit_ibm_runtime.fake_provider import FakeBrisbane 
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator, QasmSimulator
#from qiskit_aer.primitives import SamplerV2 as Sampler

from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap

num_qubits = train_features.shape[1]

backend = AerSimulator(method='statevector', device='CPU')
session = Session(backend=backend)
pass_manager = generate_preset_pass_manager(optimization_level=1, target=backend.target)

sampler = SamplerV2(mode=session)
fidelity = ComputeUncompute(sampler=sampler, pass_manager=pass_manager)

feature_map = ZZFeatureMap(feature_dimension=num_qubits)
qk = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

In [3]:
from sklearn.svm import SVC
import time

SVM = SVC(kernel=qk.evaluate)

start = time.time()
SVM.fit(train_features, train_labels)
stop = time.time()

train_time = stop - start

start = time.time()
SVM_score = SVM.score(test_features, test_labels)
stop = time.time()

test_time = stop - start

print(f"Training Time: {train_time}")
print(f"Testing Time: {test_time}")
print(f"Total Time: {train_time + test_time}")
print(f"Callable kernel classification test score: {SVM_score}")

Training Time: 32.51245474815369
Testing Time: 12.401498317718506
Total Time: 44.91395306587219
Callable kernel classification test score: 1.0
